<a href="https://colab.research.google.com/github/Suhas-S63/Deep_Learning_Lab/blob/main/DL_Lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images

melanoma-skin-cancer-dataset-of-10000-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
! unzip /content/melanoma-skin-cancer-dataset-of-10000-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: melanoma_cancer_dataset/train/benign/melanoma_643.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_644.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_645.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_646.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_647.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_648.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_649.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_65.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_650.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_651.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_652.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_653.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_654.jpg  
  inflating: melanoma_cancer_dataset/train/benign/mel

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications import VGG16
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'melanoma_cancer_dataset/train'
test_dir = 'melanoma_cancer_dataset/test'

train_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

test_data = ImageDataGenerator(rescale=1./255)


train_generator = train_data.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_data.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Print class indices
print("Class indices:", train_generator.class_indices)


Found 9605 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Class indices: {'benign': 0, 'malignant': 1}


## Custom CNN Model

In [9]:
# Custom CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## VGG16 model with custom output layers for classification

In [11]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in vgg.layers:
    layer.trainable = False

vgg_model_new_top = Sequential()

vgg_model_new_top.add(vgg)

# Add custom top layers
vgg_model_new_top.add(Flatten())
vgg_model_new_top.add(Dense(512, activation='relu'))
vgg_model_new_top.add(Dense(512, activation='relu'))
vgg_model_new_top.add(Dropout(0.5))
vgg_model_new_top.add(Dense(1, activation='sigmoid'))

In [12]:
vgg_model_new_top.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Early stopping code

This is used to stop the training of the model prematurely if the the model accuracy is not increasing after a certain epochs

In [13]:
# early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

In [ ]:
# training custom CNN model
custom_history = model.fit_generator(train_generator,
                                            steps_per_epoch=len(train_generator),
                                            epochs=10,
                                            validation_data=validation_generator,
                                            validation_steps=len(validation_generator),
                                            callbacks=[early_stop])


<ipython-input-14-e38d16c7b9a4>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  custom_history = model.fit_generator(train_generator,


Epoch 1/10
301/301 [==============================] - 143s 439ms/step - loss: 0.5255 - accuracy: 0.7312 - val_loss: 0.3097 - val_accuracy: 0.8800
Epoch 2/10
301/301 [==============================] - 121s 403ms/step - loss: 0.3599 - accuracy: 0.8508 - val_loss: 0.2793 - val_accuracy: 0.8830
Epoch 3/10
 59/301 [====>.........................] - ETA: 1:31 - loss: 0.3266 - accuracy: 0.8686

In [ ]:
# training VGG16 model with custom top layers
vgg_history = vgg_model_new_top.fit_generator(train_generator,
                                                 steps_per_epoch=len(train_generator),
                                                 epochs=10,
                                                 validation_data=validation_generator,
                                                 validation_steps=len(validation_generator),
                                                 callbacks=[early_stop])